# Eurybia - dataprep for US car accidents
This notebook describes the data preparation leading to the dataset in "US_Accidents_extract.csv", used in some of our tutorials.  


The original dataset was taken from the Kaggle [US car accidents dataset](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents).

---
Acknowledgements
- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. “A Countrywide Traffic Accident Dataset.”, 2019.
- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import category_encoders as ce

### Extract the zipped dataset if you haven't already done so

In [2]:
# from zipfile import ZipFile
# with ZipFile('/tmp/archive.zip', 'r') as zipObj:
#     zipObj.extractall()

### Load it up

In [3]:
data = pd.read_csv('/tmp/US_Accidents_Dec21_updated.csv')

In [4]:
print(data.shape)
print(data.columns)

(2845342, 47)
Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')


In [5]:
feats_to_keep = ['Start_Lat','Start_Lng','Distance(mi)','Temperature(F)','Humidity(%)','Visibility(mi)',
 'day_of_week_acc','Nautical_Twilight','season_acc','target','target_multi','year_acc','Description']

### Create targets column  
Here we regroup the severity modalities into two classes to create a binary target column : benign to moderate severity (<= 2) on one side, serious and above on the other (>2)

In [6]:
print(data.Severity.value_counts())
cond = [data.Severity <= 2]
choice = ['0']
data['target'] = np.select(cond, choice, default = '1')
data['target'].value_counts(normalize = True)*100

2    2532991
3     155105
4     131193
1      26053
Name: Severity, dtype: int64


0    89.938011
1    10.061989
Name: target, dtype: float64

In [7]:
data = data.rename(columns={'Severity':'target_multi'})

### Rework the dates  
Here we build a "day of week", a "season" and a "year" feature. This will help us detect and analyze bias or trends that occur on those timescales.  
For example, we can then measure the drift between two same seasons of consecutive years to avoid seasonal bias.  
We could also aggregate by year and mesure the drift from year to year.

In [8]:
date = ['Start_Time', 'End_Time']
for d in date:
    data[d] = pd.to_datetime(data[d])

In [9]:
data['month_acc'] = data['Start_Time'].dt.month
data['day_of_week_acc'] = data['Start_Time'].dt.dayofweek
data['year_acc'] = data['Start_Time'].dt.year
cond = [data.month_acc.isin([12,1,2]), 
       data.month_acc.isin([3,4,5]), 
       data.month_acc.isin([6,7,8]), 
       data.month_acc.isin([9,10,11])]
choix = ['winter','spring','summer','autumn']
data['season_acc'] = np.select(cond, choix, default = 'NR')

### Managing missing values  
The following short analysis, using the "year" feature we just created, lets us see how the ratio of missing values evolved over time.  

In [10]:
missing_val = pd.DataFrame()
year = np.unique(data.year_acc)
for y in year:
    sub = data[data.year_acc == y]
    missing_val_y = pd.DataFrame(sub.isnull().sum().sort_values(ascending=False)/sub.shape[0]*100)
    missing_val_y.columns = ['taux_miss_'+str(y)]
    missing_val = pd.concat([missing_val, missing_val_y], axis = 1)

In [11]:
missing_val['filtre'] = missing_val.taux_miss_2016+missing_val.taux_miss_2017+missing_val.taux_miss_2018+missing_val.taux_miss_2019+missing_val.taux_miss_2020+missing_val.taux_miss_2021
missing_val[missing_val.filtre > 0][['taux_miss_2016','taux_miss_2017','taux_miss_2018','taux_miss_2019','taux_miss_2020','taux_miss_2021']]

,taux_miss_2016,taux_miss_2017,taux_miss_2018,taux_miss_2019,taux_miss_2020,taux_miss_2021
Precipitation(in),89.891333,86.925170,85.587341,20.893606,6.218763,4.277838
Wind_Chill(F),87.970399,82.553472,77.963058,12.703439,3.912991,2.805896
Number,80.749689,80.031479,78.591827,74.640295,62.027853,53.230340
Wind_Speed(mph),17.975152,16.970070,18.992989,6.567291,3.490694,2.538126
Visibility(mi),2.394611,2.698300,2.952640,2.733020,3.012476,2.147254
Weather_Condition,2.369206,2.725143,3.046404,2.763954,2.964861,2.156647
Humidity(%),2.216777,2.360327,2.686057,2.867196,3.268442,2.266520
Temperature(F),2.090572,2.267597,2.572069,2.759314,3.066161,2.148709
Pressure(in),1.639841,1.966227,2.461146,2.200955,2.567011,1.865526
Wind_Direction,0.926867,1.159116,1.662622,3.013746,3.493251,2.538391


$\require{color}$
$\colorbox{red}{The percentage of missing values, aggregated by year, is far from constant. This is a preliminary sign of data drift.}$

In [12]:
data = data.dropna(subset=["Nautical_Twilight"])

### Final dataset features

In [13]:
data = data[feats_to_keep]

### Quantitative features  

In [14]:
for v in ['Distance(mi)','Temperature(F)','Humidity(%)','Visibility(mi)']:
    data[v] = np.round(data[v],0)

In [15]:
data['Start_Lat'] = np.round(data['Start_Lat'],1)
data['Start_Lng'] = np.round(data['Start_Lng'],1)

### Sampling  
For the purpose of our tutorials, a sample size of ~50000 is sufficient.  
The following few steps reduce the sample size down to about this number, and balance the number of samples per year, in an effort to reduce this source of bias before training a model or producing a quantitative analysis.

In [16]:
sampled_data = pd.DataFrame()
annee = np.unique(data.year_acc)
for a in annee:
    sub = data[data.year_acc == a]
    sub = sub.reset_index(drop = True)
    tir = np.random.choice(a = sub.shape[0], size = 50000//len(annee)+1, replace = False)
    sampled_data = pd.concat([sampled_data, sub.iloc[tir,:]], axis = 0)
    sampled_data = sampled_data.reset_index(drop = True)

In [17]:
sampled_data = sampled_data.iloc[np.random.choice(size = 50000, a = sampled_data.index, replace = False),:]
sampled_data = sampled_data.reset_index(drop  = True)

### Let us have a final look at our data :

In [18]:
sampled_data.head(3)

,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Visibility(mi),day_of_week_acc,Nautical_Twilight,season_acc,target,target_multi,year_acc,Description
0,33.0,-117.1,0.0,40.0,93.0,2.0,3,Day,winter,0,2,2019,At Carmel Mountain Rd - Accident.
1,29.5,-98.5,0.0,83.0,65.0,10.0,4,Day,summer,1,3,2017,At TX-345-SP/Woodlawn Ave/Exit 567B - Accident.
2,32.7,-96.8,0.0,88.0,57.0,10.0,0,Night,summer,0,2,2021,Incident on RUGGED DR near BERKLEY AVE Expect ...


In [19]:
sampled_data.describe()

,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Visibility(mi),day_of_week_acc,target_multi,year_acc
count,50000.000000,50000.000000,50000.000000,48753.000000,48682.00000,48675.000000,50000.000000,50000.000000,50000.00000
mean,37.039702,-98.101712,0.616060,61.274937,64.19005,9.188290,2.552960,2.294080,2018.49996
std,5.134426,18.374657,1.756289,18.673054,22.95887,3.063803,1.790681,0.635079,1.70787
min,24.800000,-124.500000,0.000000,-19.000000,3.00000,0.000000,0.000000,1.000000,2016.00000
25%,33.900000,-118.200000,0.000000,49.000000,48.00000,10.000000,1.000000,2.000000,2017.00000
50%,37.500000,-93.700000,0.000000,63.000000,66.00000,10.000000,2.000000,2.000000,2018.50000
75%,40.800000,-81.000000,1.000000,75.000000,83.00000,10.000000,4.000000,2.000000,2020.00000
max,49.000000,-67.800000,100.000000,118.000000,100.00000,100.000000,6.000000,4.000000,2021.00000


### Write the sample to disk

In [22]:
# sampled_data.to_csv('US_Accidents_extract.csv', index = False)